<a href="https://colab.research.google.com/github/RafaelRamos19/NPL-word2vec/blob/main/Word2vec_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dados_treino = pd.read_csv("/content/drive/MyDrive/curso_word2vac/treino.csv")
dados_teste = pd.read_csv("/content/drive/MyDrive/curso_word2vac/teste.csv")

In [ ]:
dados_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


Vamos fazer um preprocessamento com a biblioteca "spacy"
vamos instala-la logo abaixo junto com os pacotes em português

In [ ]:
!python -m spacy download pt_core_news_sm

2022-09-13 16:44:21.198694: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 3.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


verificar se o download ocorreu bem

In [ ]:
import spacy
nlp = spacy.load("pt_core_news_sm") #carregando os modelos linguísticos do porgutuês 
nlp #está tudo do jeito certo

In [ ]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)
type(doc)

spacy.tokens.doc.Doc

O nlp retornou um objeto do tipo doc
Primeiro ele faz a tokenização, depois o tag, passing... e o doc é o conjunto de tudo isso. Observe abaixo:

In [ ]:
print(doc[0])
print(doc.ents) # a entidade do texto

Rio
(Rio de Janeiro,)


ele já localizou os stop words do texto, as entidades, já colocou as tags... varias coisas já estão feitas. Podemos usar isso para tratar a nossa base de dados

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino.title) #deixando todas as palavras em minúsculo

na função vamos precisar ter textos sem stop words, ter caracteres alfabeticos e ter no mínimo 3 palavras nos títulos (o contexto). Como ela receberá um elemento tipo doc vai ser mais fácil fazer esse tratamento

In [ ]:
def trata_textos(doc):
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha # o toiken será válido quando não for stop word e for um alfabetico
    if e_valido:
      tokens_validos.append(token.text)
  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)

vamos testar a função já criada na célula abaixo:


In [ ]:
doc = nlp('Rafael Rio de Janeiro 123 e o novo')
print(trata_textos(doc))

Rafael Rio Janeiro


números, stop_words foram removidos

vamos agora fazer esse tratamento para todos os nosso títulos

O nlp estava realizando aquele tratamento com textos específicos, precisamos fazer isso com um conjunto de textos, assimn vamos usar a função: 

In [ ]:
from time import time # vamos também calcular o tempo que será necessário para fazer o processo
t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento, batch_size = 1000,
                                                         n_process = -1)] # para cada texto da nossa base de dados vamos transforma-lo num doc
tempo = time() - t0
print(tempo/60)

3.0535756826400755


vamos colocar os textos tratados dentro de um dataframe

In [ ]:
titulos_tratados = pd.DataFrame({'titulo':textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


Agora vamos realizar o treinamento do nosso modelo, para isso usaremos a biblioteca gensim

In [ ]:
from gensim.models import Word2Vec

sabemso que o word2vec pode ser treinado de duas maneiras: com o continuo_bag_of_words (cbow) ==0 e skipgram==1

O atibuto sg irá receber o número de algum dos dois

O window irá definir o tamanho do contexto, ou seja, a quantidade de palvras para serem consideradas

size será o tamanho do vetor

min_count quantidade mínima de vezes que uma palavra deve aparecer para ser levada em consideração

alpha é a taxa para chegar no valor mínimo da função de aprendizagem

min_alpha faz com que os saltos entre os testes de valores mínimos caia em cada treinamento fazendo com que aumente a chance de encontrar o valor mínimo

In [ ]:
w2v_modelo = Word2Vec(sg = 0, window = 2, size =300, min_count = 5, alpha = 0.03, min_alpha = 0.007)



observe que instanciamos o modelo porém não passamos nada, na célula abaixo vamos passar nosso vocabulário

ele irá receber uma lista com listas de tokens dos nosso títulos (essa é a regra descrita na documentação do w2v)

para que essa lista seja geradas precisamos remover as linhas vazias do nosso dataframe com os títutos

In [ ]:
titulos_tratados = titulos_tratados.dropna().drop_duplicates()

In [ ]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
w2v_modelo.build_vocab(lista_lista_tokens)

vamos agora testar o modelo com o gesim

In [ ]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

agora vamos realmente realizar o treinamento do nosso modelo

In [ ]:
w2v_modelo.corpus_count

84466

In [ ]:
w2v_modelo.train(lista_lista_tokens, total_examples=w2v_modelo.corpus_count, epochs=30 )

(14583821, 16207260)

analizando os resultados do modelo treinado

In [ ]:
w2v_modelo.most_similar("microsoft")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('unilever', 0.6151496171951294),
 ('amazon', 0.5942423343658447),
 ('tesla', 0.5337764024734497),
 ('linkedin', 0.5275835990905762),
 ('chrysler', 0.524567723274231),
 ('sky', 0.5216519832611084),
 ('viajante', 0.5158538818359375),
 ('braskem', 0.512729287147522),
 ('sony', 0.5101374983787537),
 ('lego', 0.5090205669403076)]

### treino com skipgram

In [ ]:
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)
w2v_modelo_sg.train(lista_lista_tokens, total_examples=w2v_modelo_sg.corpus_count, epochs=30 )

(14584362, 16207260)

In [ ]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.408194899559021),
 ('apple', 0.4037300944328308),
 ('android', 0.40300077199935913),
 ('buffett', 0.39123356342315674),
 ('patentes', 0.36628258228302),
 ('waze', 0.36524105072021484),
 ('yahoo', 0.36363881826400757),
 ('patente', 0.35505443811416626),
 ('anunciantes', 0.3509157598018646),
 ('chips', 0.3419954180717468)]

aparentemente o skipgram conseguiu generalizar um pouco melhor em relacao ao cbow

vamos salvar e construir um classificador e ver qual dos dois tipos tem a melhor performace

In [ ]:
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/curso_word2vac/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/curso_word2vac/modelo_skipgram.txt", binary=False)